# Dashboard - Data Processing

## Name: Rohit Dev
## Matrikel- Nr.- 411817

## Libraries Import

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
import plotly.graph_objects as go
from datetime import datetime

In [ ]:
%matplotlib inline
sns.set_style("dark")
mpl.rcParams['figure.figsize'] = (16,9)
pd.set_option('display.max_rows', 500)

## Covid-Data imported from 'Our World in Data (OWID)' as csv file
https://covid.ourworldindata.org/data/owid-covid-data.csv

In [ ]:
# Reading the raw data from 'csv file' 
data_raw = pd.read_csv('../data/raw/owid-covid-data.csv')

In [ ]:
data_raw

In [ ]:
data_raw.columns

In [ ]:
# Required data
data_required = data_raw[['date', 'location','total_cases']]

In [ ]:
data_required.tail()

In [ ]:
# Changing the column names
data_required = data_required.rename(columns={'location':'country', 'total_cases':'corona_cases'})

In [ ]:
data_required.head()

In [ ]:
# Changing date data format
date = data_required['date'][0]
type(date)

In [ ]:
date_idx = [datetime.strptime(each, '%Y-%m-%d') for each in data_required.date]

In [ ]:
data_required['date'] = date_idx

In [ ]:
data_required.head()

In [ ]:
# Display data types of columns
data_required.dtypes

In [ ]:
# Display countries list
data_required['country'].unique()

In [ ]:
data_swd = data_required[data_required['country'] == 'Sweden']
data_swd.tail()

In [ ]:
# Plot of the covid cases in Sweden over time
fig, ax1 = plt.subplots(1,1)
ax1.plot(data_swd['date'], (data_swd['corona_cases']), label='Sweden', color = 'b')

ax1.set_yscale('linear')

ax1.set_title('Covid cases over time', size = 16)
ax1.set_xlabel('Date', size = 16)
ax1.set_ylabel('Covid cases', size = 16)

ax1.legend(loc='best', prop={'size':16})

plt.grid()

## Analysis of multiple countries

In [ ]:
# Creating the dataframe with date as index
total_dates = pd.DataFrame(data_required['date'])

In [ ]:
dates_unique = total_dates['date'].unique()

In [ ]:
# Number of unique dates
len(dates_unique)

In [ ]:
df_dates = pd.DataFrame(dates_unique)

In [ ]:
df_dates.rename(columns={0:'date'}, inplace=True)   # Renames the columns and save to df_dates

In [ ]:
df_dates.sort_values(by='date', inplace=True)

In [ ]:
data_analysis = df_dates.set_index(['date'])

In [ ]:
country_list = ['India', 'Germany', 'Spain']

In [ ]:
country_list = data_required['country'].unique()

In [ ]:
# Number of unique countries
len(country_list)

# Data Processing for Dashboard

In [ ]:
# Recall raw data
data_raw.head()

## Doubling Rate

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [ ]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [ ]:
# Calculating doubling rate
pd_DR_result=data_raw[['location','total_cases']].groupby(['location']).apply(rolling_reg,'total_cases').reset_index()

In [ ]:
pd_DR_result.tail()

In [ ]:
# Changing name of the columns
pd_DR_result=pd_DR_result.rename(columns={'total_cases':'confirmed_DR',
                             'level_1':'index', 'location':'country'})
pd_DR_result

In [ ]:
# Data Frame for the confirmed covid cases only
data_required=data_required.reset_index()

In [ ]:
data_required = data_required.rename(columns={'corona_cases':'confirmed'})
data_required.head()

In [ ]:
pd_DR_result.head()

In [ ]:
# Merging the data frames: confirmed covid cases dataframe & doubling rate dataframe
pd_result_larg=pd.merge(data_required,pd_DR_result[['index','confirmed_DR']],on=['index'], how='left')

In [ ]:
pd_result_larg

In [ ]:
pd_result_larg.head()

## Filtering the data with groupby apply

In [ ]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [ ]:
data_required.tail()

In [ ]:
data_required["state"]="none"

In [ ]:
data_required.dtypes

In [ ]:
# Data Type change of Column 'confirmed' from float64 to int64...this has been done to run 'Savitzky–Golay filter' function

data_required[data_required['confirmed'].isnull()]
data_required['confirmed'] = pd.to_numeric(data_required['confirmed'], errors='coerce')
data_required = data_required.dropna(subset=['confirmed'])
data_required['confirmed'] = data_required['confirmed'].apply(np.int64)
data_required.dtypes

## Filtered confirmed covid cases

In [ ]:
pd_filtered_result=data_required[['country','confirmed']].groupby(['country']).apply(savgol_filter).reset_index()

In [ ]:
# Merging the filtered confired covid cases
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

## Filtered doubling rate

In [ ]:
pd_filtered_doubling=pd_result_larg[['country','confirmed_filtered']].groupby(['country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_1':'index'})

In [ ]:
pd_filtered_doubling.tail()

In [ ]:
# Merging the final filetered doubling rate 
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

In [ ]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [ ]:
# Exporting the processed data file
pd_result_larg.to_csv('../data/processed/Dashboard_final_set.csv',sep=';',index=False)

# End